In [1]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS.sort(key=len)

In [10]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        if not f.startswith('5-5-10-10-10-15-15'):
            continue
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 10:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
#             columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [34]:
# df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
#     df_lst.append(df[extend_df(df, long_term=True)].add_suffix('_run' + str(i)))
    extend_df(df, long_term=True)
    cols_to_keep = [i for i in df.columns if i not in ['epoch', 'lr', 't_l', 'v_3d'] and not i.startswith('usingfulltestset')]
    
    cols_to_keep = [col for col in cols_to_keep if col.split('3')[0] in ACTIONS and int(col.split('3d')[1]) in [560, 1000] or col.startswith('3')]
    
    mini_df_lst.append(df[cols_to_keep].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_22-06-2020-16:17:29.csv
1 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_23-06-2020-21:18:06.csv
2 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_22-06-2020-03:59:13.csv
3 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_23-06-2020-04:39:51.csv
4 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_21-06-2020-15:45:52.csv


In [35]:
my_cols = []

for name in LONG_TERM_ACTIONS:
    my_cols.append(name + '3d560_mean')
    my_cols.append(name + '3d1000_mean')
my_cols.append('3d560_mean')
my_cols.append('3d1000_mean')

In [36]:
tmp = get_summary_df(merged)
tmp[my_cols]

,walking3d560_mean,walking3d1000_mean,eating3d560_mean,eating3d1000_mean,smoking3d560_mean,smoking3d1000_mean,discussion3d560_mean,discussion3d1000_mean,3d560_mean,3d1000_mean
0,43.323638,43.142335,45.879971,65.254553,36.483842,62.988739,97.186821,94.650246,55.718568,66.508968


In [37]:
values = [str(i).split('.')[0] + '.' + str(i).split('.')[1][:1] for i in list(tmp[my_cols].iloc[0])]
'&'.join(values)

'43.3&43.1&45.8&65.2&36.4&62.9&97.1&94.6&55.7&66.5'